In [2]:

import numpy as np
import cv2 as cv2
import time
import datetime
from collections import deque

## Reading from your Phone's IP Camera

In [14]:

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

start_time = time.time()

fps = 0 
frame_counter = 0
 

cap = cv2.VideoCapture('http://192.168.43.48:8080/video')
 
while(True):
    
    ret, frame = cap.read()
    if not ret:
        break 
    
   
    frame_counter += 1
    fps = (frame_counter / (time.time() - start_time))
    

    cv2.putText(frame, 'FPS: {:.2f}'.format(fps), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),3)

    cv2.imshow('INTRUDER ALERT SYSTEM',frame)
    
  
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

7aYUObv2jMApJnZECNKbI5NVkxQ5uUUNj78_Y53I

In [4]:
from twilio.rest import Client
 

with open('credentials.txt', 'r') as myfile:
  data = myfile.read()
 
info_dict = eval(data)
 

account_sid = info_dict['account_sid']
 

auth_token  = info_dict['auth_token']
 

client = Client(account_sid, auth_token)
message = client.messages.create( to =info_dict['your_num'], from_ = info_dict['trial_num'], body= "Hi Buddy Right Now You are in GNI College")

## Background Subtraction

In [5]:

cap = cv2.VideoCapture('sv.mp4')


foog = cv2.createBackgroundSubtractorMOG2( detectShadows = True, varThreshold = 50, history = 2800)

while(1):
    
    ret, frame = cap.read() 
    if not ret:
        break
        
  
    fgmask = foog.apply(frame)
    

    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    

    cv2.imshow('INTRUDER ALERT SYSTEM',fgmask)
    k = cv2.waitKey(10)
    if k == 27: 
        break

cap.release()
cv2.destroyAllWindows()

## Contour Detection with Background Subtraction

In [6]:

cap = cv2.VideoCapture('sv.mp4')


kernel= None


foog = cv2.createBackgroundSubtractorMOG2( detectShadows = True, varThreshold = 50, history = 2800)


thresh = 1100

while(1):
    ret, frame = cap.read()
    if not ret:
        break
        
 
    fgmask = foog.apply(frame)
    

    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    
    fgmask = cv2.dilate(fgmask,kernel,iterations = 4)
    
  
    contours, hierarchy = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        
   
        cnt = max(contours, key = cv2.contourArea)


      
        if cv2.contourArea(cnt) > thresh:

            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(frame,'Person Detected',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)


    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    stacked = np.hstack((fgmask_3,frame))
    cv2.imshow('INTRUDER ALERT SYSTEM',cv2.resize(stacked,None,fx=0.4,fy=0.4))

    k = cv2.waitKey(40) & 0xff
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Function for sending message via Twilio

In [7]:
def send_message(body, info_dict):

    account_sid = info_dict['account_sid']

    auth_token  = info_dict['auth_token']


    client = Client(account_sid, auth_token)

    message = client.messages.create( to = info_dict['your_num'], from_ = info_dict['trial_num'], body= body)

## Function for detecting if a person is present or not

In [8]:
def is_person_present(frame, thresh=1100):
    
    global foog
    
  
    fgmask = foog.apply(frame)


    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    fgmask = cv2.dilate(fgmask,kernel,iterations = 4)

    contours, hierarchy = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
     
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > thresh:
            
         
            cnt = max(contours, key = cv2.contourArea)

       
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(frame,'Person Detected',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
            
            return True, frame
        
  
    else:
        return False, frame

## Final Application

In [9]:

cv2.namedWindow('Security Surveillance System', cv2.WINDOW_NORMAL)


cap = cv2.VideoCapture('http://192.168.43.48:8080/video')



width = int(cap.get(3))
height = int(cap.get(4))


with open('credentials.txt', 'r') as myfile:
  data = myfile.read()

info_dict = eval(data)

foog = cv2.createBackgroundSubtractorMOG2( detectShadows = True, varThreshold = 100, history = 2000)

status = False

patience = 7
detection_thresh = 15

initial_time = None

de = deque([False] * detection_thresh, maxlen=detection_thresh)

fps = 0 
frame_counter = 0
start_time = time.time()


while(True):
    
    ret, frame = cap.read()
    if not ret:
        break 
            
 
    detected, annotated_image = is_person_present(frame)  
    
    de.appendleft(detected)

    if sum(de) == detection_thresh and not status:                       
            status = True
            entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
            out = cv2.VideoWriter('outputs/{}.mp4'.format(entry_time), cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))


    if status and not detected:
     
        if sum(de) > (detection_thresh/2): 
            
            if initial_time is None:
                initial_time = time.time()
            
        elif initial_time is not None:        
            
            if  time.time() - initial_time >= patience:
                status = False
                exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                out.release()
                initial_time = None
            
                body = "Alert: \n A Person Entered the Room at {} \n Left the room at {}".format(entry_time, exit_time)
                send_message(body, info_dict)
    
    elif status and sum(de) > (detection_thresh/2):
        initial_time = None
    

    current_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")


    cv2.putText(annotated_image, 'FPS: {:.2f}'.format(fps), (510, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)

    cv2.putText(annotated_image, current_time, (700, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255),2)    
  
    cv2.putText(annotated_image, 'Room Occupied: {}'.format(str(status)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0,0,255),3)

    if initial_time is None:
        text = 'Patience: {}'.format(patience)
    else: 
        text = 'Patience: {:.2f}'.format(max(0, patience - (time.time() - initial_time)))
        
    cv2.putText(annotated_image, text, (10, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255) , 2)   

    if status:
        out.write(annotated_image)
 

    cv2.imshow('INTRUDER ALERT SYSTEM',frame)
    
    frame_counter += 1
    fps = (frame_counter / (time.time() - start_time))
    

    if cv2.waitKey(30) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
out.release()

## Sample Script for recording Testing videos

In [12]:
import numpy as np
import cv2

cap = cv2.VideoCapture('http://192.168.43.48:8080/video')


fourcc = cv2.VideoWriter_fourcc(*'XVID')
width = int(cap.get(3))
height = int(cap.get(4))


out = cv2.VideoWriter(r'sampletestnew2.mp4',fourcc, 15.0, (width, height))

while(True):
    ret, frame = cap.read()
    if not ret:
        break

 
    out.write(frame)

    cv2.imshow('INTRUDER ALERT SYSTEM',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
out.release()
cap.release()
cv2.destroyAllWindows()